In [ ]:
from torchcrf import CRF
import torch
import torch.nn as nn
from torch_optimizer import Ranger, DiffGrad
import torch.nn.functional as F
from transformers import get_cosine_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset, random_split
from utils import *
from metrics import f1score
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from multiprocessing import cpu_count
from sklearn.metrics import classification_report
from platform import system
from itertools import chain
from os import environ

environ["TOKENIZERS_PARALLELISM"] = "false"
pl.seed_everything(seed=42)

In [ ]:
LEARNING_RATE = 5e-3
BATCH_SIZE = 256
WEIGHT_DECAY = 1e-2
EPOCHS = 128
MAX_LEN = 128
N_JOBS = cpu_count() if system() != "Windows" else 0

tag2idx = {'B': 0, 'I': 1, 'O': 2, 'E': 3, 'S': 4, '<': 5, '>': 6, '$': 7}

In [ ]:
class LSTM4NER(pl.LightningModule):
    def __init__(self, 
                 input_dim, 
                 bidirectional=True, 
                 num_lstm_layers=2,
                 embed_dim=128, 
                 dropout=0.2, 
                 lstm_dim=128,
                 num_tags=len(tag2idx),
                 warmup_steps=2,
                 total_steps=1024,
                 train_dataset=None,
                 val_dataset=None,
                 test_dataset=None):

        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=input_dim,
                                      embedding_dim=embed_dim,
                                      padding_idx=0)
        

        c = (2 if bidirectional else 1)
        self.lstm = nn.LSTM(input_size=embed_dim, 
                            hidden_size=lstm_dim, 
                            dropout=dropout,
                            num_layers=num_lstm_layers, 
                            bidirectional=bidirectional)

        
        self.fc = nn.Linear(lstm_dim*c, num_tags)
        self.crf = CRF(num_tags=num_tags, batch_first=True)
        self.dropout = nn.Dropout(p=dropout*2)
        ## Hyperparameters ##
        self.learning_rate = LEARNING_RATE
        self.weight_decay = WEIGHT_DECAY
        self.batch_size = BATCH_SIZE
        self.warmup_steps = warmup_steps
        self.total_steps = total_steps
        ## Datasets ##
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.test_dataset = test_dataset

    def train_dataloader(self):
        return DataLoader(self.train_dataset, 
                          batch_size=self.batch_size,
                          shuffle=True,
                          num_workers=N_JOBS,
                          drop_last=False)


    def val_dataloader(self):
        return DataLoader(self.val_dataset, 
                          batch_size=self.batch_size,
                          num_workers=N_JOBS,
                          drop_last=False)


    def test_dataloader(self):
        return DataLoader(self.test_dataset, 
                          batch_size=self.batch_size,
                          num_workers=N_JOBS,
                          drop_last=False)


    def forward(self, input_ids):
        out = self.embedding(input_ids)
        out, _ = self.lstm(out)
        out = F.relu(out)
        out = self.dropout(out)
        out = self.fc(out)
        out = F.relu(out)
        return out


    def _shared_evaluation_step(self, batch, batch_idx):
        ids, masks, lbls = batch
        emissions = self(ids)
        loss = -self.crf(emissions, lbls, mask=masks)
        pred = self.crf.decode(emissions, mask=masks)
        r, p, f1 = f1score(lbls, pred)
        return loss, r, p, f1
     
        
    def training_step(self, batch, batch_idx):
        loss, r, p, f1 = self._shared_evaluation_step(batch, batch_idx)
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train_recall", r, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train_precision", p, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train_f1score", f1, on_step=False, on_epoch=True, prog_bar=True)
        return loss


    def validation_step(self, batch, batch_idx):
        loss, r, p, f1 = self._shared_evaluation_step(batch, batch_idx)
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("val_recall", r, on_step=False, on_epoch=True, prog_bar=True)
        self.log("val_precision", p, on_step=False, on_epoch=True, prog_bar=True)
        self.log("val_f1score", f1, on_step=False, on_epoch=True, prog_bar=True)

    
    def test_step(self, batch, batch_idx):
        loss, r, p, f1 = self._shared_evaluation_step(batch, batch_idx)
        self.log("test_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test_recall", r, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test_precision", p, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test_f1score", f1, on_step=False, on_epoch=True, prog_bar=True)


    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        ids, masks, _ = batch
        return self.crf.decode(self(ids), mask=masks)


    def configure_optimizers(self):   
        optimizer = DiffGrad(self.parameters(), 
                             lr=self.learning_rate,
                             weight_decay=self.weight_decay)

        scheduler = get_cosine_schedule_with_warmup(optimizer=optimizer,
                                                    num_warmup_steps=self.warmup_steps,
                                                    num_training_steps=self.total_steps)

        lr_scheduler = {
            'scheduler': scheduler, 
            'interval': 'step', 
            'frequency': 1
        }
        
        return [optimizer], [lr_scheduler]

In [ ]:
with open("../data/full_vocab_290818_tree_bank_tokenier.txt", mode="r", encoding="utf-8") as f:
    vocab = [s.strip() for s in f.readlines()]

In [ ]:
encoded_input, attn_masks, extended_labels = get_encoded_input("../data/train_290818.txt", 
                                                               tag2idx=tag2idx,
                                                               vocab=vocab,
                                                               maxlen=MAX_LEN)

L = len(extended_labels)

dataset = TensorDataset(torch.LongTensor(encoded_input),
                        torch.BoolTensor(attn_masks),
                        torch.LongTensor(extended_labels))

train_sz, val_sz = L-int(0.1*L), int(0.1*L)
train_dataset, val_dataset = random_split(dataset, (train_sz, val_sz))                                                                                                                      

In [ ]:
encoded_input, attn_masks, extended_labels = get_encoded_input("../data/test_290818.txt", 
                                                               tag2idx=tag2idx,
                                                               vocab=vocab,
                                                               maxlen=MAX_LEN)


test_dataset = TensorDataset(torch.LongTensor(encoded_input),
                             torch.BoolTensor(attn_masks),
                             torch.LongTensor(extended_labels))

In [ ]:
WARMUP_RATIO = 0.01
TOTAL_STEPS = len(train_dataset) // BATCH_SIZE
WARMUP_STEPS = int(WARMUP_RATIO * TOTAL_STEPS)

In [ ]:
model = LSTM4NER(input_dim=len(vocab),
                 bidirectional=True,
                 train_dataset=train_dataset,
                 val_dataset=val_dataset,
                 test_dataset=test_dataset,
                 warmup_steps=WARMUP_STEPS,
                 total_steps=TOTAL_STEPS)

earlystopping_callback = EarlyStopping(monitor="val_f1score", 
                                       min_delta=1e-4, 
                                       patience=EPOCHS, 
                                       mode="max")

checkpoint_callback = ModelCheckpoint(dirpath="./",
                                      filename=f"lstm-ner-val-f1score",
                                      save_top_k=1, 
                                      mode="max",
                                      monitor="val_f1score",
                                      save_weights_only=True)

trainer = pl.Trainer(accelerator="gpu",
                     max_epochs=EPOCHS,
                     precision=16,
                     log_every_n_steps=1,
                     callbacks=[earlystopping_callback,
                                checkpoint_callback])

In [ ]:
trainer.fit(model)

In [ ]:
model.load_state_dict(torch.load(f"./lstm-ner-val-f1score.ckpt")["state_dict"])
trainer.test(model)

In [ ]:
preds = list(chain.from_iterable(trainer.predict(model)))

preds = post_pad_sequences(preds,
                           start=tag2idx['<'], 
                           end=tag2idx['>'], 
                           pad=tag2idx['$'],
                           maxlen=len(extended_labels[0]),
                           return_masks=False)["seq"]


preds = np.array(preds).flatten()                        
lbls = np.array(extended_labels).flatten()

clf_rp = classification_report(lbls,
                               preds,
                               zero_division=0,
                               labels=[0, 1, 2, 3, 4, 5, 6, 7],
                               target_names=['B', 'I', 'O', 'E', 'S', 'START (<)', 'END (>)', 'PAD ($)'])

print("-"*80)
print(clf_rp)
print("-"*80)